In [ ]:
import requests
import json
import random
import time
import polyline
import osmnx as ox
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import LineString
from collections import defaultdict
from itertools import combinations

In [ ]:
def plot_cars_on_graph(G, cars):
    # Plot the road network first
    fig, ax = ox.plot_graph(G, node_color='black', node_size=5, edge_linewidth=0.5, bgcolor ='white', show=False, close=False,)

    # Extract coordinates separately for batch plotting
    src_lats = [car['src_coords'][0] for car in cars]
    src_lons = [car['src_coords'][1] for car in cars]
    dst_lats = [car['dst_coords'][0] for car in cars]
    dst_lons = [car['dst_coords'][1] for car in cars]

    # Plot origins (green circles)
    ax.scatter(src_lons, src_lats, c='green', marker='o', s=30, label='Origin', zorder=3)

    # Plot destinations (red Xs)
    ax.scatter(dst_lons, dst_lats, c='red', marker='x', s=30, label='Destination', zorder=3)

    # Optional: connect each OD pair with a line
    for car in cars:
        ax.plot(
            [car['src_coords'][1], car['dst_coords'][1]],
            [car['src_coords'][0], car['dst_coords'][0]],
            color='blue', linewidth=1, alpha=0.5
        )

    # Add legend and title
    ax.legend()
    plt.title("Car Origins (green) and Destinations (red)")
    plt.show()

In [ ]:
plot_cars_on_graph(G, CARS)

In [ ]:
from collections import defaultdict

def compute_route_overlap_congestion(car_routes, precision=5):
    """
    Calculates congestion score based on overlapping route points,
    ensuring each car contributes only once per point.

    Returns:
        congestion_scores: dict {(car_id, route_index): congestion_score}
        point_freq: dict {point: number of unique cars using the point}
    """
    point_to_cars = defaultdict(set)

    # Step 1: For each car, register unique points it visits (across all its routes)
    for car in car_routes:
        car_id = car['car_id']
        car_points = set()
        for route in car.get('routes', []):
            for lat, lon in route['geometry']:
                key = (round(lat, precision), round(lon, precision))
                car_points.add(key)
        for key in car_points:
            point_to_cars[key].add(car_id)

    # Step 2: Build a frequency map of unique cars per point
    point_freq = {key: len(cars) for key, cars in point_to_cars.items()}

    # Step 3: For each route, compute average congestion
    congestion_scores = {}
    for car in car_routes:
        car_id = car['car_id']
        for idx, route in enumerate(car.get('routes', [])):
            total = 0
            for lat, lon in route['geometry']:
                key = (round(lat, precision), round(lon, precision))
                total += point_freq.get(key, 0)
            avg_congestion = total / max(len(route['geometry']), 1)
            congestion_scores[(car_id, idx)] = avg_congestion

    return congestion_scores, point_freq


In [ ]:
CONGESTION_SCORES, POINT_FREQ = compute_route_overlap_congestion(CAR_ROUTES)
print(CONGESTION_SCORES)

In [ ]:
import folium
from folium import Map, PolyLine, Marker
from branca.colormap import linear

def visualize_routes_by_overlap_congestion(car_routes, congestion_scores):
    if not car_routes:
        print("No routes to display.")
        return None

    # Center map on the first available route
    for car in car_routes:
        if car.get("routes"):
            center = car["origin"]
            break
    else:
        print("⚠️ No cars have routes.")
        return None

    fmap = folium.Map(location=center, zoom_start=13, tiles='cartodbpositron')

    # Normalize congestion scores for colormap
    all_scores = list(congestion_scores.values())
    colormap = linear.YlOrRd_09.scale(min(all_scores), max(all_scores))
    colormap.caption = "Route Overlap-Based Congestion"
    fmap.add_child(colormap)

    for car in car_routes:
        car_id = car['car_id']
        routes = car.get('routes', [])
        if not routes:
            continue

        # Add start and end markers
        #Marker(location=car['origin'], popup=f"Car {car_id} Start").add_to(fmap)
        #Marker(location=car['destination'], popup=f"Car {car_id} End").add_to(fmap)

        for idx, route in enumerate(routes):
            poly = route['geometry']
            dist_m = route.get('distance', 0)
            time_sec = route.get('duration', 0)
            score = congestion_scores.get((car_id, idx), 0)
            color = colormap(score)

            tooltip_text = (
                f"Car {car_id} - Route {idx}<br>"
                f"Distance: {dist_m / 1000:.2f} km<br>"
                f"Duration: {time_sec // 60:.1f} min<br>"
                f"Congestion Score: {score:.2f}"
            )

            PolyLine(
                locations=poly,
                color=color,
                weight=4,
                opacity=0.7,
                tooltip=tooltip_text
            ).add_to(fmap)

    return fmap


In [ ]:
# Step 1: Compute congestion scores
congestion_scores, _ = compute_route_overlap_congestion(CAR_ROUTES)

# Step 2: Visualize map
map_overlap = visualize_routes_by_overlap_congestion(CAR_ROUTES, congestion_scores)
map_overlap.save("routes_by_overlap_congestion.html")


In [ ]:
def calculate_weights_from_congestion_scores(congestion_scores):
    """
    Computes adjusted weights w(i,j,k) by subtracting self-overlap from congestion scores.

    Args:
        congestion_scores: dict {(i, k): score}

    Returns:
        weights[i][j][k]: shared weight from congestion (ignoring self-use)
    """
    weights = defaultdict(lambda: defaultdict(dict))
    
    all_keys = congestion_scores.keys()
    cars = sorted(set(i for i, _ in all_keys))
    ks = sorted(set(k for _, k in all_keys))

    for i, j in combinations(cars, 2):
        for k in ks:
            if (i, k) in congestion_scores and (j, k) in congestion_scores:
                score_i = max(0, congestion_scores[(i, k)] - 1)
                score_j = max(0, congestion_scores[(j, k)] - 1)
                avg = (score_i + score_j) / 2
                weights[i][j][k] = avg
                weights[j][i][k] = avg
            else:
                weights[i][j][k] = 0
                weights[j][i][k] = 0

    return weights


In [ ]:
weights = calculate_weights_from_congestion_scores(congestion_scores)
print(weights)


In [ ]:
import networkx as nx
from collections import defaultdict
from itertools import combinations

def normalize_point(p, precision=5):
    """Round a lat/lon point to the given decimal precision."""
    return (round(p[0], precision), round(p[1], precision))

def normalize_segment(segment, precision=5):
    a = normalize_point(segment[0], precision)
    b = normalize_point(segment[1], precision)
    return (a, b)  # direction matters now!


def build_car_overlap_graph(car_routes, precision=5):
    """
    car_routes: dict of car_id -> list of route alternatives (each route is a list of segments)
    Each segment is a tuple: ((lat1, lon1), (lat2, lon2))
    """
    segment_to_cars = defaultdict(set)

    for car_id, routes in car_routes.items():
        for route in routes:
            for segment in route:
                norm_seg = normalize_segment(segment, precision)
                segment_to_cars[norm_seg].add(car_id)

    # Initialize graph
    G = nx.Graph()
    overlap_counts = defaultdict(int)

    for segment, cars in segment_to_cars.items():
        for car1, car2 in combinations(cars, 2):
            pair = tuple(sorted((car1, car2)))
            overlap_counts[pair] += 1

    for (car1, car2), weight in overlap_counts.items():
        G.add_edge(car1, car2, weight=weight)

    return G


In [ ]:
def cluster_cars(overlap_graph):
    """
    Returns: list of sets of car_ids, each set is a cluster
    """
    return list(nx.connected_components(overlap_graph))


In [ ]:
def transform_routes_to_segments(car_routes_raw):
    def to_segments(points):
        return [(points[i], points[i + 1]) for i in range(len(points) - 1)]

    transformed = {}
    for car in car_routes_raw:
        car_id = car['car_id']
        routes = [to_segments(route['geometry']) for route in car['routes']]
        transformed[car_id] = routes
    return transformed

In [ ]:
# Use it like this:
car_routes_segments = transform_routes_to_segments(CAR_ROUTES)
print(car_routes_segments)
OVERLAP_GRAPH = build_car_overlap_graph(car_routes_segments)
print(OVERLAP_GRAPH)
L_ = cluster_cars(OVERLAP_GRAPH)
print(L_)

In [ ]:
####### ILP implementation

from collections import defaultdict

def compute_directional_segment_frequencies(car_routes_segments):
    segment_to_cars = defaultdict(set)

    for car_id, routes in car_routes_segments.items():
        for route in routes:
            for segment in route:
                segment_to_cars[segment].add(car_id)

    segment_freq = {seg: len(cars) for seg, cars in segment_to_cars.items()}
    return segment_freq

def compute_congestion_scores(car_routes_segments, segment_freq):
    congestion_scores = {}

    for car_id, routes in car_routes_segments.items():
        for idx, route in enumerate(routes):
            score = sum(segment_freq.get(seg, 0) for seg in route)
            avg_score = score / max(len(route), 1)
            congestion_scores[(car_id, idx)] = avg_score
    return congestion_scores


def calculate_weights_from_congestion_scores(congestion_scores):
    weights = defaultdict(lambda: defaultdict(dict))
    
    all_keys = congestion_scores.keys()
    cars = sorted(set(i for i, _ in all_keys))
    ks = sorted(set(k for _, k in all_keys))

    for i, j in combinations(cars, 2):
        for k in ks:
            if (i, k) in congestion_scores and (j, k) in congestion_scores:
                score_i = max(0, congestion_scores[(i, k)] - 1)
                score_j = max(0, congestion_scores[(j, k)] - 1)
                avg = (score_i + score_j) / 2
                weights[i][j][k] = avg
                weights[j][i][k] = avg
    return weights


from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD
from itertools import combinations

def solve_cluster_ilp_with_aux_vars(cluster_car_ids, k, weights):
    """
    ILP for assigning routes to cars with accurate congestion weights.
    Uses auxiliary variables to model x[i,r] * x[j,r].
    """
    prob = LpProblem("RouteAssignmentWithAux", LpMinimize)

    # 1. Binary decision variables: x[i][r] = 1 if car i chooses route r
    x = {(i, r): LpVariable(f"x_{i}_{r}", cat=LpBinary)
         for i in cluster_car_ids for r in range(k)}

    # 2. Constraints: Each car must choose exactly one route
    for i in cluster_car_ids:
        prob += lpSum(x[i, r] for r in range(k)) == 1

    # 3. Auxiliary variables: z[i][j][r] = 1 if both i and j choose route r
    z = {}
    for i, j in combinations(cluster_car_ids, 2):
        for r in range(k):
            z[(i, j, r)] = LpVariable(f"z_{i}_{j}_{r}", cat=LpBinary)

            # Add constraints to model z_ijr = x_ir * x_jr
            prob += z[(i, j, r)] <= x[i, r]
            prob += z[(i, j, r)] <= x[j, r]
            prob += z[(i, j, r)] >= x[i, r] + x[j, r] - 1

    # 4. Objective: minimize weighted overlap (accurate!)
    terms = []
    for (i, j, r), var in z.items():
        w = weights.get(i, {}).get(j, {}).get(r, 0)
        terms.append(w * var)

    prob += lpSum(terms)

    # 5. Solve
    prob.solve(PULP_CBC_CMD(msg=False))

    # 6. Extract solution
    result = {}
    for i in cluster_car_ids:
        for r in range(k):
            if x[i, r].varValue == 1:
                result[i] = r
    return result



In [ ]:
segment_freq = compute_directional_segment_frequencies(car_routes_segments)
congestion_scores = compute_congestion_scores(car_routes_segments, segment_freq)
weights = calculate_weights_from_congestion_scores(congestion_scores)

K_ALTERNATIVES = len(next(iter(car_routes_segments.values())))  # assumes same k for all


In [ ]:
#solutions = []
#for cluster in L_:
    #car_ids = list(cluster)
    #print(car_ids)
    #print(K_ALTERNATIVES)
    #print(weights)
    #result = solve_cluster_ilp_with_aux_vars(car_ids, K_ALTERNATIVES, weights)
    #solutions.append(result)
    #print(f"Cluster with {len(car_ids)} cars → {result}")

In [ ]:
from itertools import combinations

def solve_cluster_greedy(cluster_car_ids, k, weights):
    """
    Greedy route assignment: assign each car a route that minimizes its added overlap
    based on already assigned cars using congestion weights.

    Args:
        cluster_car_ids: list of car IDs in the cluster
        k: number of route alternatives per car
        weights: dict[i][j][r] where r is route index, and weights are symmetric

    Returns:
        result: dict of {car_id: selected_route_index}
    """
    assigned = {}  # {car_id: route_idx}

    for i in cluster_car_ids:
        best_score = float('inf')
        best_route = 0

        for r in range(k):
            score = 0

            # Accumulate interaction cost with already assigned cars
            for j in assigned:
                rj = assigned[j]
                # Prefer symmetric access to weights
                w_ij = weights.get(i, {}).get(j, {}).get(r, 0)
                w_ji = weights.get(j, {}).get(i, {}).get(rj, 0)
                score += w_ij + w_ji

            if score < best_score:
                best_score = score
                best_route = r

        assigned[i] = best_route

    return assigned


In [ ]:
solutions = []
for cluster in L_:
    car_ids = list(cluster)
    print(car_ids)
    #print(K_ALTERNATIVES)
    #print(weights)
    result = solve_cluster_greedy(car_ids, K_ALTERNATIVES, weights)
    solutions.append(result)
    print(f"Cluster with {len(car_ids)} cars → {result}")

In [ ]:
## Comparision

def compute_segment_frequencies_from_all_routes(car_routes_segments):
    from collections import defaultdict
    freq = defaultdict(int)

    for routes in car_routes_segments.values():
        for route in routes:
            for seg in route:
                freq[seg] += 1
    return freq

segment_freq_before = compute_segment_frequencies_from_all_routes(car_routes_segments)

def compute_segment_frequencies_from_assignment(car_routes_segments, assignment):
    from collections import defaultdict
    freq = defaultdict(int)

    for car_id, route_idx in assignment.items():
        route = car_routes_segments[car_id][route_idx]
        for seg in route:
            freq[seg] += 1
    return freq

# Merge all cluster results
final_assignment = {}
for cluster_result in solutions:  # from ILP loop
    final_assignment.update(cluster_result)

segment_freq_after = compute_segment_frequencies_from_assignment(car_routes_segments, final_assignment)

def compare_congestion_stats(freq_before, freq_after):
    total_before = sum(freq_before.values())
    total_after = sum(freq_after.values())

    unique_segments_before = len(freq_before)
    unique_segments_after = len(freq_after)

    max_before = max(freq_before.values()) if freq_before else 0
    max_after = max(freq_after.values()) if freq_after else 0

    return {
        "Total uses before": total_before,
        "Total uses after": total_after,
        "Unique segments before": unique_segments_before,
        "Unique segments after": unique_segments_after,
        "Max congestion before": max_before,
        "Max congestion after": max_after,
        "Δ Total": total_before - total_after,
        "Δ Max congestion": max_before - max_after,
    }

congestion_stats = compare_congestion_stats(segment_freq_before, segment_freq_after)
for k, v in congestion_stats.items():
    print(f"{k}: {v}")


In [ ]:
import folium
from folium import Map, PolyLine, Marker
from branca.colormap import linear

def visualize_assigned_routes(car_routes, final_assignment, congestion_scores):
    if not car_routes or not final_assignment:
        print("No data to display.")
        return None

    # Center map on the first car with a route
    for car in car_routes:
        if car.get("routes"):
            center = car["origin"]
            break
    else:
        print("⚠️ No routes found.")
        return None

    fmap = folium.Map(location=center, zoom_start=13, tiles='cartodbpositron')

    # Normalize only the selected congestion scores
    selected_scores = [
        congestion_scores.get((car['car_id'], final_assignment[car['car_id']]), 0)
        for car in car_routes if car['car_id'] in final_assignment
    ]
    colormap = linear.YlOrRd_09.scale(min(selected_scores), max(selected_scores))
    colormap.caption = "Selected Route Congestion"
    fmap.add_child(colormap)

    for car in car_routes:
        car_id = car['car_id']
        if car_id not in final_assignment:
            continue

        routes = car.get('routes', [])
        route_idx = final_assignment[car_id]

        if route_idx >= len(routes):
            continue  # Safety check

        route = routes[route_idx]
        poly = route['geometry']
        dist_m = route.get('distance', 0)
        time_sec = route.get('duration', 0)
        score = congestion_scores.get((car_id, route_idx), 0)
        color = colormap(score)

        #Marker(location=car['origin'], popup=f"Car {car_id} Start").add_to(fmap)
        #Marker(location=car['destination'], popup=f"Car {car_id} End").add_to(fmap)

        tooltip_text = (
            f"Car {car_id} - Route {route_idx}<br>"
            f"Distance: {dist_m / 1000:.2f} km<br>"
            f"Duration: {time_sec // 60:.1f} min<br>"
            f"Congestion Score: {score:.2f}"
        )

        PolyLine(
            locations=poly,
            color=color,
            weight=5,
            opacity=0.75,
            tooltip=tooltip_text
        ).add_to(fmap)

    return fmap


In [ ]:
# Combine all cluster assignments into one dict
final_assignment = {}
for cluster_result in solutions:  # from your ILP loop
    final_assignment.update(cluster_result)

# Now visualize only selected routes
map_ = visualize_assigned_routes(CAR_ROUTES, final_assignment, congestion_scores)
map_.save("optimized_routes_map.html")
